# Word Count using DF

- You already implemented the word count application using RDD. Now implement the same using DF

### Instructions
1. Study the below code, understand it.
2. Apply your understanding and write down the word count application in DataFrame to count the frequecy of occurance of each word in descending order, from a file `constitution.txt` residing in your home on HDFS.
3. Note down every API you used in the solution and have a brainstorming session with your coding partner to solidify your understanding.

- **TimeLine** - 45 Mins


In [2]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [3]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [1]:
stop_words = ['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 'her',
 'hers',
 'herself',
 'it',
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each',
 'few',
 'more',
 'most',
 'other',
 'some',
 'such',
 'no',
 'nor',
 'not',
 'only',
 'own',
 'same',
 'so',
 'than',
 'too',
 'very',
 'can',
 'will',
 'just',
 'don',
 'should',
 'now',
  '']


In [2]:
!pwd

/home/talentum/spark-jupyter/Surprise_Lab_WordCount


In [8]:
import pyspark.sql.functions as F

df1 = spark.read.text("hdfs:///user/talentum/constitution.txt")
print(df1.printSchema())


root
 |-- value: string (nullable = true)

None


In [13]:
type(df1)

pyspark.sql.dataframe.DataFrame

In [15]:
#df1.show(5, truncate = False)
df1.show(10)
df2 = df1.withColumn("split", F.split(F.col('value'), "\s+"))
df2.show(10)
df2.drop(F.col('value')).select(F.explode(F.col('split')).alias("Word")).show(10)
# .filter(~F.col('Word').isin(stop_words)).groupBy(F.col('Word')).count() \
# .sort(F.col('count').desc()).withColumnRenamed('count', 'Freq')
# df2.show(20, truncate = False)
                     

+--------------------+
|               value|
+--------------------+
|We the People of ...|
|Union, establish ...|
|common defence, p...|
|Liberty to oursel...|
|Constitution for ...|
|Article 1.  Secti...|
|vested in a Congr...|
|Senate and House ...|
|Representatives s...|
|the People of the...|
+--------------------+
only showing top 10 rows

+--------------------+--------------------+
|               value|               split|
+--------------------+--------------------+
|We the People of ...|[We, the, People,...|
|Union, establish ...|[Union,, establis...|
|common defence, p...|[common, defence,...|
|Liberty to oursel...|[Liberty, to, our...|
|Constitution for ...|[Constitution, fo...|
|Article 1.  Secti...|[Article, 1., Sec...|
|vested in a Congr...|[vested, in, a, C...|
|Senate and House ...|[Senate, and, Hou...|
|Representatives s...|[Representatives,...|
|the People of the...|[the, People, of,...|
+--------------------+--------------------+
only showing top 10 rows

+-------+


In [12]:
#df1.show(5, truncate = False)
# df1.show(10)
df2 = df1.withColumn("split", F.split(F.col('value'), "\s+")) \
.drop(F.col('value')).select(F.explode(F.col('split')).alias("Word")) \
.filter(~F.col('Word').isin(stop_words)).groupBy(F.col('Word')).count() \
.sort(F.col('count').desc()).withColumnRenamed('count', 'Freq')
df2.show(20, truncate = False)
                     

+---------------+----+
|Word           |Freq|
+---------------+----+
|shall          |293 |
|United         |85  |
|President      |72  |
|The            |64  |
|States,        |55  |
|State          |47  |
|States         |46  |
|may            |42  |
|Congress       |39  |
|Amendment      |28  |
|House          |27  |
|President,     |27  |
|Vice           |24  |
|one            |24  |
|Representatives|22  |
|Section        |22  |
|No             |21  |
|State,         |20  |
|person         |20  |
|two            |19  |
+---------------+----+
only showing top 20 rows



In [14]:
#df1.show(5, truncate = False)
# df1.show(10)
df2 = df1.withColumn("split", F.split(F.col('value'), "\s+")) \
.drop(F.col('value')).select(F.explode(F.col('split')).alias("Word")) \
.groupBy(F.col('Word')).count() \
.sort(F.col('count').desc()).withColumnRenamed('count', 'Freq')
df2.show(20, truncate = False)
                     

+---------+----+
|Word     |Freq|
+---------+----+
|the      |662 |
|         |645 |
|of       |493 |
|shall    |293 |
|and      |256 |
|to       |183 |
|be       |178 |
|or       |157 |
|in       |137 |
|by       |100 |
|a        |94  |
|United   |85  |
|for      |81  |
|any      |79  |
|President|72  |
|as       |64  |
|The      |64  |
|have     |63  |
|States,  |55  |
|such     |52  |
+---------+----+
only showing top 20 rows

